In [2]:
using LaTeXStrings
import PyPlot; plt=PyPlot
plt.rc("figure", dpi=300.0)
plt.rc("figure", figsize=(6,4))
# plt.rc("figure", autolayout=true)
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
cbbPalette = ["#E69F00", "#56B4E9", "#009E73", "#F0E442", "#0072B2", "#D55E00", "#CC79A7"]
;

In [38]:
using DataFrames
# import PDMats: AbstractPDMat, PDMat
# import Base: mean
using Distributions
# using Base.LinAlg
# using Distances
# using GeoJSON
# using LibGEOS
using GLM
# import MultivariateStats
# import StatsModels
# mvst=MultivariateStats
;

In [4]:
include("../src/GeoRDD.jl")


Use "MultiGPCovars{MT,KT1,KT2}(...) where {MT,KT1,KT2}" instead.


GeoRDD

# Dataset and Preprocessing

In [5]:
module NYC
    include("NYC.jl")
end
include("NYCPlots.jl")

NYC_sales = NYC.read_processed_sales()

filtered = NYC.filter_sales(NYC_sales)

sentinels = NYC.read_sentinels()

model_dicts = NYC.sales_dicts(filtered[:filtered])
schdistr_indices = model_dicts[:schdistr_indices]
schdistrs = model_dicts[:schdistrs]
X_dict = model_dicts[:X_dict]
Y_dict = model_dicts[:Y_dict]
;

d = NYC_sales[.!ismissing.(NYC_sales[:SchDistr]), :]
distr1 = d[:SchDistr].=="01"
res = [NYC.RESIDENTIAL_DICT[v] for v in d[NYC.BUILDING_CLASS_CATEGORY]]
sqft = .!ismissing.(d[NYC.SQFT])
d[res .& sqft .& distr1, :]

missing_sqft = d[res .& .!sqft, :]
size(missing_sqft)

INFO: Recompiling stale cache file /Users/imolk/Library/Julia/packages_experimental/lib/v0.6/Formatting.ji for module Formatting.
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #Source#24(::String, ::CSV.Options{Void}, ::Int64, ::Int64, ::Dict{String,Union}, ::Symbol, ::Bool, ::Bool, ::Bool, ::Int64, ::Int64, ::Int64, ::Bool, ::Type{T} where T) at /Users/imolk/Library/Julia/packages_experimental/v0.6/CSV/src/Source.jl:66
 [3] (::Core.#kw#Type)(::Array{Any,1}, ::Type{CSV.Source}) at ./<missing>:0
 [4] #Source#23(::UInt8, ::UInt8, ::UInt8, ::String, ::Void, ::Int64, ::Int64, ::Dict{String,Union}, ::Symbol, ::Bool, ::Void, ::UInt8, ::String, ::String, ::Bool, ::Bool, ::Int64, ::Int64, ::Int64, ::Bool, ::Type{T} where T, ::String) at /Users/imolk/Library/Julia/packages_experimental/v0.6/CSV/src/Source.jl:30
 [5] (::Core.#kw#Type)(::Array{Any,1}, ::Type{CSV.Source}, ::String) at ./<missing>:0
 [6] #read#43(::Bool, ::Dict{Int64,Function}, ::Bool, ::Array{Any,1}, ::Fu

District 01 has 4 sales
District 02 has 80 sales
District 03 has 31 sales
District 04 has 6 sales
District 05 has 39 sales
District 06 has 30 sales
District 07 has 69 sales
District 08 has 739 sales
District 09 has 163 sales
District 10 has 312 sales
District 11 has 1172 sales
District 12 has 204 sales
District 13 has 321 sales
District 14 has 195 sales
District 15 has 516 sales
District 16 has 420 sales
District 17 has 459 sales
District 18 has 559 sales
District 19 has 686 sales
District 20 has 1010 sales
District 21 has 770 sales
District 22 has 1453 sales
District 23 has 245 sales
District 24 has 1283 sales
District 25 has 1317 sales
District 26 has 1268 sales
District 27 has 2199 sales
District 28 has 1033 sales
District 29 has 1956 sales
District 30 has 695 sales
District 31 has 0 sales
District 32 has 344 sales


(27394, 18)

# model matrix

In [21]:
local_distrs = ["18","19","23","24","25","26","27","28","29"]
local_indices = vcat(collect(schdistr_indices[distr] for distr in local_distrs)...)
;

In [90]:
for_mm = DataFrame(
        Y=float.(filtered[:filtered][:logSalePricePerSQFT]),
        BUILDING=filtered[:filtered][NYC.BUILDING_CLASS_AT_TIME_OF_SALE],
        TAX=filtered[:filtered][NYC.TAX_CLASS_AT_TIME_OF_SALE],
        DISTRICT=string.(filtered[:filtered][:SchDistr]),
)
size(for_mm)

(19578, 4)

In [91]:
NYC_ols = lm(@formula(Y ~ BUILDING + DISTRICT), 
    for_mm[local_indices, :],
    )

StatsModels.DataFrameRegressionModel{GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Formula: Y ~ 1 + BUILDING + DISTRICT

Coefficients:
                Estimate Std.Error   t value Pr(>|t|)
(Intercept)      5.60448 0.0218029   257.052   <1e-99
BUILDING: A1  -0.0466165 0.0146214  -3.18824   0.0014
BUILDING: S1   -0.120938  0.051939  -2.32846   0.0199
BUILDING: A9  -0.0246849 0.0319052 -0.773697   0.4391
BUILDING: A2    0.116118 0.0182145     6.375    <1e-9
BUILDING: B1   -0.228897 0.0178637  -12.8135   <1e-36
BUILDING: S2   -0.390796 0.0379945  -10.2856   <1e-23
BUILDING: B3  -0.0748821 0.0170774  -4.38487    <1e-4
BUILDING: B9   -0.257676 0.0341961  -7.53524   <1e-13
BUILDING: B2   -0.184555 0.0167834  -10.9963   <1e-27
BUILDING: C0   -0.355748 0.0194462   -18.294   <1e-72
BUILDING: A4  -0.0319171     0.217 -0.147083   0.8831
BUILDING: A3    0.143069 0.0478621   2.98919   0.0028
BUILDING: S0   -

In [92]:
NYC_coefdict = Dict(zip(coefnames(NYC_ols.mf), coef(NYC_ols)))

NYC_coefdict["DISTRICT: 19"] - NYC_coefdict["DISTRICT: 27"]

-0.1160648046201865

**Use district 27 as base.**

In [98]:
methods(StatsModels.EffectsCoding)

# 4 methods for generic function "(::Type)":
StatsModels.EffectsCoding(; base, levels) in StatsModels at /Users/imolk/Library/Julia/packages_experimental/v0.6/StatsModels/src/contrasts.jl:212
StatsModels.EffectsCoding(base, levels::Union{Array{T,1} where T, Void}) in StatsModels at /Users/imolk/Library/Julia/packages_experimental/v0.6/StatsModels/src/contrasts.jl:208
StatsModels.EffectsCoding(base, levels) in StatsModels at /Users/imolk/Library/Julia/packages_experimental/v0.6/StatsModels/src/contrasts.jl:208
(::Type{T})(arg) where T in Base at sysimg.jl:77

In [104]:
StatsModels.EffectsCoding()

StatsModels.EffectsCoding(nothing, nothing)

In [110]:
mf = StatsModels.ModelFrame(@formula(Y ~ BUILDING + DISTRICT), 
    for_mm,
    contrasts = Dict(:DISTRICT => StatsModels.EffectsCoding("27", nothing)),
    )
mm = StatsModels.ModelMatrix(mf)

NYC_ols_27 = lm(mm.m, for_mm[:Y]) # use district 27 as base

NYC_coefdict_27 = Dict(zip(coefnames(mf), coef(NYC_ols_27)))

NYC_coefdict_27["DISTRICT: 19"]

-0.5303669662620432